In [ ]:
# Simulated Object class

from dataclasses import dataclass
from typing import Optional

from dynamics.solver import DynamicsSolver

import torch

@dataclass
class SimulatedObject():
    mass: float
    coordinates: tuple
    velocity: tuple
    acceleration: tuple
    solver: DynamicsSolver
    save_past: int = 0
    past_coordinates: list = None
    past_velocity: list = None
    past_acceleration: list = None
    nb_updates: int = 0

    def update(self, dt: float, force: Optional[tuple] = None):
        if self.save_past > 0:
            if self.past_coordinates is None:
                self.past_coordinates = []
            if self.past_velocity is None:
                self.past_velocity = []
            if self.past_acceleration is None:
                self.past_acceleration = []
        
            self.past_coordinates.append(self.coordinates)
            self.past_velocity.append(self.velocity)
            self.past_acceleration.append(self.acceleration)

            if len(self.past_coordinates) > self.save_past:
                self.past_coordinates.pop(0)
                self.past_velocity.pop(0)
                self.past_acceleration.pop(0)

        print(self.nb_updates, "x: ", self.coordinates, "v: ", self.velocity, "a: ", self.acceleration, "f: ", force)
        x, v = self.solver.apply_force(x=torch.tensor(self.coordinates).view(1,1,-1), v=torch.tensor(self.velocity).view(1,1,-1), force=torch.tensor(force).view(1,1,-1), dt=dt)
        self.coordinates = tuple(x[0,-1,:].tolist())
        self.velocity = tuple(v[0,-1,:].tolist())
        self.acceleration = tuple([f / self.mass for f in force])
        self.nb_updates += 1

    def get_past_coordinates(self, dt: Optional[float] = None) -> tuple:
        if self.past_coordinates is None:
            return [self.coordinates]
        
        if dt is None:
            dt = len(self.past_coordinates)
        
        return self.past_coordinates[-dt:] + [self.coordinates]
    
    def get_past_velocity(self, dt: Optional[float] = None) -> tuple:
        if self.past_velocity is None:
            return [self.velocity]

        if dt is None:
            dt = len(self.past_velocity)

        return self.past_velocity[-dt:] + [self.velocity]

    

In [ ]:
# Agent class

from model.dynamics_model import DYNAMIC_MODELS

# model_type = "dynamical_variational"
model_type = "dynamical_lstm"
model_save = "meerkat_modelling/sksurxrm/checkpoints/epoch=19-step=14400.ckpt"

dynamical_predictor = DYNAMIC_MODELS[model_type].load_from_checkpoint(checkpoint_path=model_save)

dynamical_predictor.eval()
torch.no_grad()


def dynamical_agent(state: SimulatedObject) -> tuple:
    x = state.get_past_coordinates()
    v = state.get_past_velocity()
    x = torch.tensor(x).unsqueeze(0).to(dynamical_predictor.device)
    v = torch.tensor(v).unsqueeze(0).to(dynamical_predictor.device)
    if x.size(1) < 2:
        x = x.repeat(1, 2, 1)
        v = v.repeat(1, 2, 1)
    
    force = dynamical_predictor(x,velocity=v)
    force = tuple(force[0,-1,:].tolist())

    return force
    


In [ ]:
# Simulation parameters

minX, maxX = -1, 2
minY, maxY = -1, 2
minV, maxV = -0.005, 0.005
maxT = 200
nbObjects = 0
nbGravObjects = 0
nbAgents = 1
nbGtAgents = 1
mass = 1
gravity_field = -9.81 * 1e-5
tau = 5
ground_truth = True

assert (nbGtAgents == 0 and not ground_truth) or (nbGtAgents > 0 and ground_truth)

In [ ]:
# Set ground truth movements

if ground_truth:

        from data.format_data import PandasFormatter
        from data.dataset import SeriesDataset
        import numpy as np
        import pandas as pd

        data_file = "data/test/22-10-20_C2_20.csv"

        formatter = PandasFormatter(pd.read_csv(data_file))
        sequences = formatter.format(output_format="dataclass").movements
        sequences = {ind : coords.to_numpy(dtype=np.float64).tolist() for ind, coords in sequences.items()}

        too_short_sequences = [ind for ind, seq in sequences.items() if len(seq) < maxT]
        for ind in too_short_sequences:
                del sequences[ind]
        assert len(sequences) >= nbGtAgents, f"Number of sequences ({len(sequences)}) is less than the number of ground truth agents ({nbGtAgents})"

        # mean_std_coords = tuple([(np.array(val).mean(), np.array(val).std()) for val in zip(*[sample for seq in sequences.values() for sample in seq])])  # mean and std values for each dimension
        # mean_coord_t = torch.tensor([mean_std_coords[0][0], mean_std_coords[1][0], mean_std_coords[2][0]])
        # std_coord_t = torch.tensor([mean_std_coords[0][1], mean_std_coords[1][1], mean_std_coords[2][1]])
        min_coord_x, max_coord_x, min_coord_y, max_coord_y, min_coord_z, max_coord_z = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for seq in sequences.values():
                for sample in seq:
                        min_coord_x = min(min_coord_x, sample[0])
                        max_coord_x = max(max_coord_x, sample[0])
                        min_coord_y = min(min_coord_y, sample[1])
                        max_coord_y = max(max_coord_y, sample[1])
                        min_coord_z = min(min_coord_z, sample[2])
                        max_coord_z = max(max_coord_z, sample[2])
        min_coords = torch.tensor([min_coord_x, min_coord_y, min_coord_z])
        max_coords = torch.tensor([max_coord_x, max_coord_y, max_coord_z])

        solver = DynamicsSolver(mass=mass, dimensions=3)

        def transform(x_i, x_ip1, prev_v0 = None):
                x = torch.tensor(x_i).float()
                y = torch.stack([torch.tensor(x_i).float(), torch.tensor(x_ip1).float()]) # concatenate x_i and x_i+1

                prev_v0 = torch.tensor(prev_v0).float().view(1,1,-1) if prev_v0 is not None else None

                # x = (x - mean_coord_t) / std_coord_t # normalize data
                # y = (y - mean_coord_t) / std_coord_t # normalize data
                x = (x - min_coords) / (max_coords - min_coords) # normalize data
                y = (y - min_coords) / (max_coords - min_coords) # normalize data
                
                force, a, v = solver.compute_force(y.unsqueeze(0), v0=prev_v0, return_velocity=True) # target data is force applied on target step (t+1), corresponds to acceleration when setting mass=1
                
                y = force[:,0,:].squeeze(0) # force applied on step x_i to reach x_i+1
                v0 = v[:,-1,:].squeeze(0) # velocity reached at step x_i+1

                return x, y, v0

        trajectories = {}
        initial_states = {}
        authorised_agents = []
        for ind, seq in sequences.items():
                if ind not in authorised_agents and len(authorised_agents) < nbGtAgents:
                        authorised_agents.append(ind)

                if ind in authorised_agents:
                        v0 = None # Assume initial speed is 0
                        for i in range(len(seq) - 1):
                                x, y, v0 = transform(seq[i], seq[i+1], v0)
                                
                                tx = tuple(x.tolist())
                                ta = tuple(y.tolist())
                                v0 = tuple(v0.tolist())

                                if ind not in trajectories:
                                        trajectories[ind] = []
                                trajectories[ind].append(ta)

                                if ind not in initial_states:
                                        initial_states[ind] = (tx,ta)
        
        class GtForceAttributor():
                def __init__(self, trajectories):
                        self.trajectories = trajectories
                        self.individuals = list(trajectories.keys())
                        self.nb_individuals = len(self.individuals)
                        self.idx = [0] * self.nb_individuals
                        self.current_ind = 0

                def __call__(self, state: SimulatedObject) -> tuple:
                        force = self.trajectories[self.individuals[self.current_ind]][self.idx[self.current_ind]]

                        self.idx[self.current_ind] += 1
                        self.current_ind = (self.current_ind + 1) % self.nb_individuals

                        return force
                
        gt_force_attributor = GtForceAttributor(trajectories)
        def ground_truth_agent(state: SimulatedObject) -> tuple:
                return gt_force_attributor(state)

In [ ]:
# Test trajectories

individual_ids = list(trajectories.keys())
print("individual_ids", individual_ids)

err = 1e-6

for i in range(len(individual_ids)):
    trajectories_i = trajectories[individual_ids[i]]
    sequences_i = sequences[individual_ids[i]]
    print(f"Individual {i} has {len(sequences_i)} / {len(trajectories_i)} steps")

    vj0 = [0.0, 0.0, 0.0]
    min_x = sequences_i[0]
    max_x = sequences_i[0]
    for j in range(len(sequences_i)-1):
        xj0 = sequences_i[j]
        xj1 = sequences_i[j+1]

        min_x = [min(min_x[k], xj0[k]) for k in range(3)]
        max_x = [max(max_x[k], xj0[k]) for k in range(3)]

        # xj0 = ((torch.tensor(xj0).float() - mean_coord_t) / std_coord_t).tolist()
        # xj1 = ((torch.tensor(xj1).float() - mean_coord_t) / std_coord_t).tolist()
        xj0 = ((torch.tensor(xj0).float() - min_coords) / (max_coords - min_coords)).tolist()
        xj1 = ((torch.tensor(xj1).float() - min_coords) / (max_coords - min_coords)).tolist()

        force = trajectories_i[j]
        retrieved_xj1, retrieved_vj1 = solver.apply_force(x=torch.tensor(xj0).view(1,1,-1), v=torch.tensor(vj0).view(1,1,-1), force=torch.tensor(force).view(1,1,-1), dt=1)
        retrieved_xj1 = retrieved_xj1[0,-1,:].tolist()
        vj0 = retrieved_vj1[0,-1,:].tolist()

        assert abs(retrieved_xj1[0] - xj1[0]) < err, f"Error on xj1[0] for individual {i} at step {j}: {retrieved_xj1[0]} != {xj1[0]} (allowed error: {err})"

    print(f"Individual {i} has a trajectory between {min_x} and {max_x}")


In [ ]:
# Set forces

import math

def bounceWall(object: SimulatedObject, dimension: int, is_max: bool, wall_coordinate: int):
    force = [0,0]
    if is_max and object.coordinates[dimension] > wall_coordinate:
        force[dimension] = -2 * object.mass * abs(object.velocity[dimension])
    elif not is_max and object.coordinates[dimension] < wall_coordinate:
        force[dimension] = 2 * object.mass * abs(object.velocity[dimension])
    return force + [0]

def gravity(object: SimulatedObject, gravity_field: float):
    return (0, gravity_field * object.mass, 0)

def radialGravity(object: SimulatedObject, gravity_field: float):
    center = (maxX + minX) / 2, (maxY + minY) / 2
    angle = math.atan2(object.coordinates[1] - center[1], object.coordinates[0] - center[0])
    return (math.cos(angle) * gravity_field * object.mass, math.sin(angle) * gravity_field * object.mass, 0)


object_forces = [
    lambda object, dt: bounceWall(object, 0, True, maxX),
    lambda object, dt: bounceWall(object, 0, False, minX),
    lambda object, dt: bounceWall(object, 1, True, maxY),
    lambda object, dt: bounceWall(object, 1, False, minY)
]

gravobject_forces = object_forces + [lambda object, dt: radialGravity(object, gravity_field)]

agent_forces = [lambda object, dt: dynamical_agent(object)]

if ground_truth:
    gtagents_forces = [lambda object, dt: ground_truth_agent(object)]

In [ ]:
# Set initial states

if ground_truth:
    objects = [SimulatedObject(
        mass=mass,
        coordinates=initial_states[authorised_agents[i % len(authorised_agents)]][0],
        velocity=tuple([0.0, 0.0, 0.0]),
        acceleration=initial_states[authorised_agents[i % len(authorised_agents)]][1],
        solver=DynamicsSolver(mass=mass,dimensions=3),
        save_past=tau,
    ) for i in range(nbObjects)]
    gravobjects = [SimulatedObject(
        mass=mass,
        coordinates=initial_states[authorised_agents[i % len(authorised_agents)]][0],
        velocity=tuple([0.0, 0.0, 0.0]),
        acceleration=initial_states[authorised_agents[i % len(authorised_agents)]][1],
        solver=DynamicsSolver(mass=mass,dimensions=3),
        save_past=tau,
    ) for i in range(nbGravObjects)]
    agents = [SimulatedObject(
        mass=mass,
        coordinates=initial_states[authorised_agents[i % len(authorised_agents)]][0],
        velocity=tuple([0.0, 0.0, 0.0]),
        acceleration=initial_states[authorised_agents[i % len(authorised_agents)]][1],
        solver=DynamicsSolver(mass=mass,dimensions=3),
        save_past=tau,
    ) for i in range(nbAgents)]
    gtagents = [SimulatedObject(
        mass=mass,
        coordinates=initial_states[authorised_agents[i % len(authorised_agents)]][0],
        velocity=tuple([0.0, 0.0, 0.0]),
        acceleration=initial_states[authorised_agents[i % len(authorised_agents)]][1],
        solver=DynamicsSolver(mass=mass,dimensions=3),
        save_past=tau,
    ) for i in range(nbGtAgents)]
    individuals = objects + gravobjects + agents + gtagents

else:
    individuals = [SimulatedObject(
        mass=mass,
        coordinates=(np.random.uniform(minX, maxX), np.random.uniform(minY, maxY), 0),
        velocity=(np.random.uniform(minV, maxV), np.random.uniform(minV, maxV), 0),
        acceleration=(0, 0, 0),
        solver=DynamicsSolver(mass=mass,dimensions=3),
        save_past=tau,
    ) for _ in range(nbObjects + nbGravObjects + nbAgents + nbGtAgents)]

    objects = individuals[:nbObjects]
    gravobjects = individuals[nbObjects:nbObjects+nbGravObjects]
    agents = individuals[nbObjects+nbGravObjects:nbObjects+nbGravObjects+nbAgents]
    gtagents = individuals[nbObjects+nbGravObjects+nbAgents:]

In [ ]:
%matplotlib notebook

# Run and display simulation

import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np

fig, ax = plt.subplots()
ax.axis([minX, maxX, minY, maxY])
lo,     = ax.plot([], [], "ro")
lgo,    = ax.plot([], [], "bo")
la,     = ax.plot([], [], "go")
lga,    = ax.plot([], [], "yo") 


def animate(i):
    for i, individual in enumerate(individuals):
        force = [0, 0, 0]
        if i < nbObjects:
            forces = object_forces
        elif i < nbObjects + nbGravObjects:
            forces = gravobject_forces
        elif i < nbObjects + nbGravObjects + nbAgents:
            forces = agent_forces
        elif i < nbObjects + nbGravObjects + nbAgents + nbGtAgents:
            forces = gtagents_forces
        else:
            raise ValueError("Too many individuals")

        for f in forces:
            dx, dy, dz = f(individual, 1)
            force[0] += dx
            force[1] += dy
        individual.update(1, tuple(force))
    
    lo.set_data([object.coordinates[0] for object in objects], [object.coordinates[1] for object in objects])
    lgo.set_data([gravobject.coordinates[0] for gravobject in gravobjects], [gravobject.coordinates[1] for gravobject in gravobjects])
    la.set_data([agent.coordinates[0] for agent in agents], [agent.coordinates[1] for agent in agents])
    lga.set_data([gtagent.coordinates[0] for gtagent in gtagents], [gtagent.coordinates[1] for gtagent in gtagents])
    
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=maxT, interval=10)

from IPython.display import HTML
HTML(ani.to_jshtml())